In [23]:
!pip install numpy opencv-python matplotlib scikit-learn tensorflow keras


In [25]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA


In [27]:
# Load the VGG16 model pretrained on ImageNet dataset
base_model = VGG16(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)  # Extract features from the 'fc1' layer


553467096/553467096 ━━━━━━━━━━━━━━━━━━━━ 48s 0us/step


In [28]:
def extract_features(img_path, model):
    img = load_img(img_path, target_size=(224, 224))  # Resize image
    img_array = img_to_array(img)  # Convert image to array
    img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions
    img_array = preprocess_input(img_array)  # Preprocess for VGG16
    features = model.predict(img_array)  # Extract features
    return features.flatten()  # Flatten to 1D


In [40]:
image_folder = r"C:\Users\aryai\OneDrive\Pictures\Screenshots"
# Change to your folder path
image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder) if img.endswith(('.png', '.jpg', '.jpeg'))]

features_list = []
for img_path in image_paths:
    features = extract_features(img_path, model)
    features_list.append(features)

features_array = np.array(features_list)  # Convert to NumPy array


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [44]:
num_samples, num_features = features_array.shape
n_components = min(100, num_samples)  # Ensure PCA does not exceed available samples

pca = PCA(n_components=n_components)
features_array_pca = pca.fit_transform(features_array)


In [46]:
def find_similar_images(query_img_path, model, image_paths, features_array, top_n=5):
    query_features = extract_features(query_img_path, model)  # Extract query image features
    query_features_pca = pca.transform([query_features])  # Apply PCA transformation
    
    similarities = cosine_similarity(query_features_pca, features_array)  # Compute cosine similarity
    similar_indices = similarities.argsort()[0][-top_n:][::-1]  # Get top N similar images
    
    # Display results
    fig, axes = plt.subplots(1, top_n + 1, figsize=(15, 5))
    axes[0].imshow(cv2.imread(query_img_path)[..., ::-1])
    axes[0].set_title("Query Image")
    axes[0].axis('off')

    for i, idx in enumerate(similar_indices):
        img = cv2.imread(image_paths[idx])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        axes[i + 1].imshow(img)
        axes[i + 1].set_title(f"Similar {i+1}")
        axes[i + 1].axis('off')

    plt.show()


In [ ]:
query_image_path = ""C:\Users\aryai\OneDrive\Pictures\IMG-20240110-WA0089.jpg""
find_similar_images(query_image_path, model, image_paths, features_array_pca)
